In [15]:
import numpy as np
import pandas as pd
import os
os.chdir('/home/vasste/ml')
from telecom.train_columns import *
os.chdir('/home/vasste/mlbootcap/TelecomDataCup/dataset')

c = pd.read_pickle('train/subs_csi_train.pk')
f = pd.read_pickle('train/subs_features_train.pk')
f['COM_CAT#34'] = f['COM_CAT#34'].fillna(f.describe()['COM_CAT#34']['75%'], axis=0)
f['COM_CAT#34'] = f['COM_CAT#34'].astype(int)

# ['COM_CAT#1', 'COM_CAT#34', 'COM_CAT#2', 'COM_CAT#3', 'INTERNET_TYPE_ID', 'DEVICE_TYPE_ID', 'COM_CAT#25', 'COM_CAT#26', 'ACT']:

j = 'COM_CAT#8'
f = f.join(c)
target = 'CSI'
df = f
f[j] = f[j].astype('category')
lst = []
for i in range(df[j].nunique()):
    val = list(df[j].unique())[i]
    lst.append([j,                                                        # Variable
                val,                                                            # Value
                df[df[j] == val].count()[i],                        # All
                df[(df[j] == val) & (df[target] == 0)].count()[j],  # Good (think: Fraud == 0)
                 df[(df[j] == val) & (df[target] == 1)].count()[j]]) # Bad (think: Fraud == 1)

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))
print(data)
    


IndexError: index out of bounds